In [13]:
import os
import configparser

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()
config.read('../config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

In [14]:
import assignment
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate #import the Class for creating a prompt

parser = StrOutputParser()

actor = assignment.actor_picker() 
location = assignment.location_picker()
theme = assignment.theme_picker()

system_template = "generates a story centered around the chosen Actor, Location, and Theme.Please provide a {responseType} response." 

input = f"Create a story where {actor} is the main character in {location}, and the story follows a {theme} theme."
# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", input)
    ]
)

# prompt Template also implements runnable and can be easily chained.
model = ChatCohere(model="command-r-plus")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})


"Ranbir Kapoor, a young and charismatic businessman, finds himself in the vibrant city of Kolkata on a business trip that would change his life forever. As he stepped out of the airport, the bustling city with its unique blend of modern and historic architecture captivated him. \n\nOn his way to the hotel, Ranbir's cab passed through the iconic Victoria Memorial, its white marble dome shining in the evening sun. The cabbie, noticing Ranbir's curiosity, shared stories of the city's rich cultural heritage and its famous literary and artistic legacy. \n\nThe next day, while attending a business meeting in the heart of Kolkata, Ranbir's path crossed with a beautiful and enigmatic young woman named Sonali. She was a local artist, passionate about painting and the city's culture. Their eyes met, and an instant spark ignited between them. \n\nAs fate would have it, they found themselves stuck in an elevator together due to a sudden power outage. In the dimly lit elevator, they introduced them

In [15]:
model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})

'Here\'s a story for you:\n\nTitle: Love in the City of Joy\n\nRanbir Kapoor, a successful businessman, had always been drawn to the vibrant city of Kolkata. There was something about the streets, the food, and the people that fascinated him. So, when his company offered him a chance to expand its operations in the city, he jumped at the opportunity.\n\nAs Ranbir settled into his new life in Kolkata, he began to explore the city, immersing himself in its rich cultural heritage. One day, while strolling through the bustling streets of College Street, he stumbled upon a quaint little bookstore. The sign above the door read "The Bookworm\'s Paradise". Intrigued, Ranbir pushed open the door and stepped inside.\n\nThe store was a treasure trove of books, with shelves upon shelves of novels, classics, and rare manuscripts. As Ranbir browsed through the shelves, he noticed a young woman, with striking features and piercing green eyes, sitting behind the counter. She introduced herself as Nand

In [16]:
# import the chat history classes
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio # library for writing code that interacts with DB, network calls etc. 

#Create a store in memory
store = InMemoryChatMessageHistory()


# Lets define a function that gets messages from store
async def getMessage():
    await asyncio.sleep(2) # this will mimic a read from DB
    print("Messages retrieved from DB")
    return await store.aget_messages()

# Add the initial messages to the store
store.add_message(HumanMessage(input))

# Retrieve the stored messages 
messages = await getMessage()

# Model invocation to generate a story based on the prompt
response = model.invoke(messages)  # Call the model with the messages
print("Generated Story:", response.content)  # Display the generated story

# Add the model's response to the store
store.add_message(SystemMessage(response.content))

# allow the user to customize the story

user_input = "surprising twist at the end"
user_input1 = f"change the given story according to user input {user_input}"

# Add the user's customization request to the store
store.add_message(HumanMessage(user_input1))

# Retrieve updated messages after user input
messages = await getMessage()

# Display all stored messages
print("Conversation History:", messages)

# Call the model again with the updated conversation including the user's input
response = model.invoke(messages)
print("Updated Story:", response.content)

# Add the updated model response to the store
store.add_message(SystemMessage(response.content))

Messages retrieved from DB
Generated Story: The city of joy, Kolkata, was bustling with life as Ranbir Kapoor stepped out of the iconic Esplanade Metro Station. He had just finished a long day of shooting for his latest film and was looking forward to a relaxing evening. As he walked down the busy streets, the aroma of jhal muri and kochuri wafted through the air, making his stomach growl with hunger.

As he turned a corner, he spotted a small café tucked away in a quiet alley. The sign above the door read "Kalyan's Café" and the words "Homely Food" were scribbled in bold letters. Ranbir's eyes lit up as he pushed open the door and was greeted by the warm aroma of freshly brewed coffee.

He took a seat at a small table by the window and perused the menu, his eyes widening at the variety of dishes on offer. Just as he was about to order, a girl with long, curly brown hair and bright green eyes walked in, her eyes scanning the café as if searching for someone. Ranbir's heart skipped a be